In [2]:
import pandas as pd
import yfinance as yf
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns
from custom_dataset import UnivariateSeriesDataModule
from torchinfo import summary

In [3]:
sp500 = yf.download("^GSPC", period="max", start="2001-1-1")
sp500.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-02,1320.280029,1320.280029,1276.050049,1283.270020,1283.270020,1129400000
2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000
2001-01-04,1347.560059,1350.239990,1329.140015,1333.339966,1333.339966,2131000000
2001-01-05,1333.339966,1334.770020,1294.949951,1298.349976,1298.349976,1430800000
2001-01-08,1298.349976,1298.349976,1276.290039,1295.859985,1295.859985,1115500000


In [3]:
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("lightning_logs")
N_LAGS = 7
HORIZON = 1

In [4]:
from typing import Any
import torch
from torch import nn
from torch.nn import functional as F


class UnivariateLSTM(pl.LightningModule):
    def __init__(
        self,
        input_dim: int = 1,
        hidden_dim: int = 128,
        num_layers: int = 2,
        output_dim: int = 1,
    ) -> None:
        super().__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.hidden_dim).to(
            self.device
        )
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.hidden_dim).to(
            self.device
        )
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        loss = F.mse_loss(y_pred, y[0])
        self.log("train_loss", loss, batch_size=self.batch_size)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        loss = F.mse_loss(y_pred, y[0])
        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        loss = F.mse_loss(y_pred, y[0])
        self.log("test_loss", loss)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_pred = self(x["encoder_cont"])
        return y_pred

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [5]:
datamodule = UnivariateSeriesDataModule(
    data=sp500,
    n_lags=N_LAGS,
    horizon=HORIZON,
    batch_size=32,
    test_size=0.2,
    feature_name="Close",
)

In [6]:
model = UnivariateLSTM(input_dim=1, hidden_dim=128, num_layers=2, output_dim=1)

In [7]:
model

UnivariateLSTM(
  (lstm): LSTM(1, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [8]:
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, RichProgressBar

early_stop_callback = EarlyStopping(
    monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min"
)
model_checkpoint = ModelCheckpoint(
    dirpath="checkpoints", filename="{epoch}-{val_loss:.2f}-{other_metric:.2f}"
)
rich_progress_bar = RichProgressBar()

In [9]:
trainer = pl.Trainer(
    max_epochs=30,
    logger=logger,
    callbacks=[early_stop_callback, model_checkpoint, rich_progress_bar],
)
trainer.fit(model, datamodule=datamodule)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/lightning_logs


┏━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name ┃ Type   ┃ Params ┃
┡━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━┩
│ 0 │ lstm │ LSTM   │  199 K │
│ 1 │ fc   │ Linear │    129 │
└───┴──────┴────────┴────────┘

Trainable params: 199 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 199 K                                                                                                
Total estimated model params size (MB): 0

Output()

/Users/raopend/Workspace/stock-index-prediction/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/data
.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any 
miscalculations, use `self.log(..., batch_size=batch_size)`.

`Trainer.fit` stopped: `max_epochs=30` reached.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir trainer.log_dir

In [10]:
trainer.test(model, datamodule=datamodule)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.16355687379837036    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.16355687379837036}]

In [11]:
trainer.validate(model=model, datamodule=datamodule)

Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │   0.0017783504445105791   │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.0017783504445105791}]

In [12]:
prediction = trainer.predict(model=model, datamodule=datamodule)

Output()